# **Preliminary Analysis ETL Northwind**

*   Nama : Irvan Septiadi
*   NPM : 065118204




Penambahan *Library* pendukung

In [ ]:
!pip install --upgrade plotly

     |████████████████████████████████| 13.2MB 280kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


# **Import Library**

Pada analisis saya menggunakan beberapa *library* untuk mengolah data

In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import psycopg2 as pg
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# **Membuat Koneksi Dengan Database di Heroku**

Database yang saya gunakan pada heroku adalah database *northwind* dan saya menggunakan tabel *orders*

In [ ]:
connectionString = {
    "user" : "pzyrxqrvijxbfu",
    "password" : "f43539fc19ac0a7f04d5e39baef357b23e9a96552c7d6702956a2b24e4aab587",
    "host" : "ec2-54-198-252-9.compute-1.amazonaws.com",
    "port" : 5432,
    "dbname" : "df34lsrrhseeov"
}

engine = pg.connect(**connectionString)

In [ ]:
df_orders = pd.read_sql("select * from orders", con=engine)
df_orders["order_date"] = pd.to_datetime(df_orders["order_date"])
print(df_orders.head())
print(df_orders.tail())
print(df_orders.info())

   order_id customer_id  employee_id  ... ship_region ship_postal_code ship_country
0     10248       VINET            5  ...        None            51100       France
1     10249       TOMSP            6  ...        None            44087      Germany
2     10250       HANAR            4  ...          RJ        05454-876       Brazil
3     10251       VICTE            3  ...        None            69004       France
4     10252       SUPRD            4  ...        None           B-6000      Belgium

[5 rows x 14 columns]
     order_id customer_id  ...  ship_postal_code ship_country
825     11073       PERIC  ...             05033       Mexico
826     11074       SIMOB  ...              1734      Denmark
827     11075       RICSU  ...              1204  Switzerland
828     11076       BONAP  ...             13008       France
829     11077       RATTC  ...             87110          USA

[5 rows x 14 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 830 entries, 0 to 829
Data c

# **Univariate Analysis**

Pada analisis ini saya akan menampilkan negara yang paling banyak melakukan pemesanan dalam database *northwind* yang saya gunakan.

In [ ]:
orders_countries = df_orders.copy()
orders_countries = orders_countries.groupby("ship_country").count()
orders_countries = orders_countries["order_id"]
orders_countries = orders_countries.reset_index()
orders_countries = orders_countries.sort_values(by="order_id")

In [ ]:
fig = px.bar(orders_countries, x="order_id", y="ship_country", color="order_id")
fig.show()

Pada grafik visualisasi diatas dapat dilihat bahwa negara yang paling banyak melakukan pemesanan merupakan negara Amerika Serikat. untuk melihat hasil negara dengan pemesanan terbesar dapat dilakukan agregasi sebagai berikut :

In [ ]:
orders_countries2 = orders_countries.copy()
orders_countries2.loc[orders_countries2["order_id"] < 80, "ship_country"] = "Other"
orders_countries2 = orders_countries2.groupby("ship_country").sum().reset_index()

In [ ]:
fig = px.pie(orders_countries2, values="order_id", names="ship_country", labels={"ship_country" : "ship_country"})
fig.update_traces(textposition="inside", textinfo="percent+label")
fig.show()

# **Bivariate Analysis**

In [ ]:
order_stats = df_orders.copy()
order_stats = order_stats[["order_id", "ship_country", "order_date"]]
order_stats = order_stats.set_index('order_date')
order_stats = order_stats.groupby('ship_country').resample('M').sum()
order_stats = order_stats.reset_index()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(name="Germany", mode='lines+markers',
                         x=order_stats[order_stats["ship_country"] == 'Germany'].order_date,
                         y=order_stats[order_stats["ship_country"] == 'Germany'].order_id))
fig.add_trace(go.Scatter(name="USA", mode='lines+markers',
                         x=order_stats[order_stats["ship_country"] == 'USA'].order_date,
                         y=order_stats[order_stats["ship_country"] == 'USA'].order_id))
fig.add_trace(go.Scatter(name="Brazil", mode='lines+markers',
                         x=order_stats[order_stats["ship_country"] == 'Brazil'].order_date,
                         y=order_stats[order_stats["ship_country"] == 'Brazil'].order_id))
fig.show()

# **Multivariate Analysis**

Pada analisis ini saya membandingkan jumlah pengiriman di setiap negara dengan jasa penyedia kargo, dan dari hasil analisis ini dapat diketahui jasa penyedia kargo mana yang memiliki popularitas tertinggi di setiap negara.

In [ ]:
df_shippers = pd.read_sql('SELECT * FROM shippers', con=engine)
df_orders_enriched = df_orders.merge(df_shippers, left_on="ship_via", right_on="shipper_id")
pd_orders_shipment = df_orders_enriched.pivot_table(index="ship_country", columns="company_name", values="freight", aggfunc=np.sum)
pd_orders_shipment

company_name,Federal Shipping,Speedy Express,United Package
ship_country,,,
Argentina,55.54,131.97,411.07
Austria,2107.10,2218.57,3065.83
Belgium,279.73,269.00,731.41
Brazil,823.84,1542.92,2513.43
Canada,848.29,284.10,1065.70
Denmark,390.44,400.84,604.91
Finland,233.30,278.01,399.58
France,1823.37,1149.98,1264.49
Germany,3646.99,3671.96,3964.33


In [ ]:
pd_orders_shipment_transposed = pd_orders_shipment.T
dict_heatmap = {
    'z': pd_orders_shipment_transposed.values.round(2).tolist(),
    'x': pd_orders_shipment_transposed.columns.tolist(),
    'y': pd_orders_shipment_transposed.index.tolist()
}

fig = ff.create_annotated_heatmap(**dict_heatmap, annotation_text=dict_heatmap["z"])
fig.show()